In [20]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

In [21]:
def getData():
    folder = "/home/tbrownex/data/test/"
    file   = "autoEncoder.csv"
    df = pd.read_csv(folder+file,index_col=0)
    cols = ["v1","v2","v3","v4","v5","v6","v7","v8","v9","label"]
    df.columns = cols
    return df

In [22]:
def prepData(df):
    train, test = train_test_split(df, test_size=0.2)
    train = train[train['label'] == 0]     # Train on "normal" data only
    del train["label"]
    del test["label"]
    
    scaler = MinMaxScaler()
    train = scaler.fit_transform(train)
    test  = scaler.transform(test)
    train, val = train_test_split(train, test_size=0.1)
    return train, val, test

In [23]:
df = getData()
trainData, valData, testData = prepData(df)

/home/tbrownex/tensorflow/lib/python3.5/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [24]:
feature_count = trainData.shape[1]

In [25]:
# NN hyper-parameters
inputSize   = feature_count
l1Size      = 6
l2Size      = 3
l3Size      = 2
l4Size      = l2Size
l5Size      = l1Size
outputSize  = inputSize

lr          = .0002
LAMBDA      = 10e-5          # Regularization parameter
batchSize   = 50
epochs      = 100
activation  = ['tanh']           # 'tanh' 'leakyReLU' 'ReLU' 'relu6' 'elu' 'crelu'
STD         = [0.2]

actf = tf.nn.tanh

In [26]:
# Set up the network
tf.reset_default_graph()
X  = tf.placeholder("float", shape=[None, feature_count], name="input")

l1w = tf.Variable(tf.truncated_normal([feature_count, l1Size], stddev=STD, dtype=tf.float32))
l2w = tf.Variable(tf.truncated_normal([l1Size, l2Size], stddev=STD, dtype=tf.float32))
l3w = tf.Variable(tf.truncated_normal([l2Size, l3Size], stddev=STD, dtype=tf.float32))
l4w = tf.Variable(tf.truncated_normal([l3Size, l4Size], stddev=STD, dtype=tf.float32))
l5w = tf.Variable(tf.truncated_normal([l4Size, l5Size], stddev=STD, dtype=tf.float32))
l6w = tf.Variable(tf.truncated_normal([l5Size, feature_count], stddev=STD, dtype=tf.float32))

l1b = tf.Variable(tf.zeros(l1Size))
l2b = tf.Variable(tf.zeros(l2Size))
l3b = tf.Variable(tf.zeros(l3Size))
l4b = tf.Variable(tf.zeros(l4Size))
l5b = tf.Variable(tf.zeros(l5Size))
l6b = tf.Variable(tf.zeros(feature_count))

l1Out = actf(tf.matmul(X,l1w)+l1b)
l2Out = actf(tf.matmul(l1Out,l2w)+l2b)
l3Out = actf(tf.matmul(l2Out,l3w)+l3b)
l4Out = actf(tf.matmul(l3Out,l4w)+l4b)
l5Out = actf(tf.matmul(l4Out,l5w)+l5b)
output = actf(tf.matmul(l5Out,l6w)+l6b)

In [27]:
mse      = tf.reduce_mean(tf.square(output - X))
L1Layer1 = LAMBDA**tf.reduce_sum(tf.abs(l1w))
L2Layer1 = LAMBDA*tf.nn.l2_loss(l1w)
    
cost = tf.reduce_mean(mse + L1Layer1 + L2Layer1)

optimizer = tf.train.AdamOptimizer(lr)
train     = optimizer.minimize(cost)

In [28]:
init=tf.global_variables_initializer()

In [29]:
num_training_batches = int(trainData.shape[0] / batchSize)
print("{} training batches of size {}".format(num_training_batches, batchSize))

64 training batches of size 50


In [34]:
with tf.Session() as sess:
    print("Epoch   Validation Loss")
    sess.run(init)
    
    for i in range(epochs):
        data = shuffle(trainData)
        for j in range(num_training_batches):
            x_mini = trainData[j*batchSize:j*batchSize+batchSize]
            _ = sess.run(train, feed_dict = {X: x_mini})
        if i % 5 == 0:
            valLoss = sess.run(mse, feed_dict={X:valData})
            print("{:<8}{:.3f}".format(i, valLoss))
    recreated = sess.run(output, feed_dict = {X: valData})

Epoch   Validation Loss
0       0.505
5       0.190
10      0.049
15      0.034
20      0.030
25      0.028
30      0.028
35      0.027
40      0.027
45      0.027
50      0.027
55      0.027
60      0.027
65      0.027
70      0.027
75      0.027
80      0.027
85      0.027
90      0.027
95      0.026


In [41]:
recreated[0].round(2)

array([0.55, 0.92, 0.63, 0.46, 0.01, 0.78, 0.58, 0.75, 0.61],
      dtype=float32)

In [40]:
valData[0].round(2)

array([0.63, 1.  , 0.69, 0.78, 0.  , 0.87, 0.26, 0.94, 0.28])